# Numba vs Cython for Implementing CMR Operations
> A basic comparison of Cython and Numba for speeding up operations central to simulation of the Context Maintenance and Retrieval (CMR) model

cymr, an implementation of the Context Maintenance and Retrieval (CMR) model, leverages Cython for faster execution compared to base Python. While Cython's speedup is essential given CMR's computational demands, its specialized syntax and concepts present a barrier for users or contributors unfamiliar with low-level programming languages or preferring regular Python.

This notebook highlights the substantial speedup Cython offers for cymr operations. However, it also demonstrates an alternative method to achieve these gains using Numba, a tool that compiles Python functions "just in time" into optimized machine code. Unlike Cython, Numba doesn't demand specialized syntax or additional compilation steps, making it more accessible to users with different skill levels. Even for experienced Cython users, Numba might be easier to use and maintain.

The focus here is on the function `integrate_context` from `cymr`'s operations.pyx, with a direct comparison of Cython and Python versions, and the application of Numba's `@njit` decorator to the Python translation. The results reveal that Numba-compiled Python code is substantially faster than both Cython and base Python versions, proving that Numba is a promising alternative to Cython for CMR. Thus, Numba can enhance speed while avoiding Cython's accessibility challenges.

## `integrate_context`
The function `integrate_context` in `operations.pyx` uses a hidden function `calc_rho` to integrate contextual input into a context vector.
We import the module for experiments here, but also copy the underlying code here for reference.

In [15]:
from operations import integrate_context as cython_integrate_context

### Implementation Using Cython

```cython
@cython.profile(False)
cdef inline double calc_rho(double cdot, double B):
    """
    Calculate context integration scaling factor.
    
    Parameters
    ----------
    cdot
        Dot product between :math:`c` and :math:`c^{IN}`.
    
    B
        Beta parameter weighting :math:`c^{IN}`.
    
    Returns
    -------
    rho
        Scaling factor for :math:`c`.   
    """
    rho = sqrt(1 + (B * B) * ((cdot * cdot) - 1)) - (B * cdot)
    return rho


cpdef integrate_context(double [:] c, double [:] c_in, double B, int [:] c_ind):
    """
    Integrate context input.
    
    Parameters
    ----------
    c
        Context state :math:`c`.
    
    c_in
        Input to context :math:`c^{IN}`
    
    B
        :math:`\beta` parameter weighting :math:`c`.
    
    c_ind
        Start and end indices of context to update.
    """
    cdef double cdot = 0
    cdef int i
    for i in range(c_ind[0], c_ind[1]):
        cdot += c[i] * c_in[i]
    rho = calc_rho(cdot, B)

    for i in range(c_ind[0], c_ind[1]):
        c[i] = rho * c[i] + B * c_in[i]
```

### Implementation Using Base Numpy
Unlike in Cython, type annotations here are optional when it comes to getting a function going -- both here, and even once we start using Numba.
Type annotation is useful anyway, but in this case we exclude them to show off how little is required to prototype a function that runs as fast as the Cython version.

In [16]:
import numpy as np

def calc_rho(cdot, B):
    rho = np.sqrt(1 + (B * B) * ((cdot * cdot) - 1)) - (B * cdot)
    return rho

def integrate_context(c, c_in, B, c_ind):
    cdot = 0
    
    for i in range(c_ind[0], c_ind[1]):
        cdot += c[i] * c_in[i]
    rho = calc_rho(cdot, B)

    for i in range(c_ind[0], c_ind[1]):
        c[i] = rho * c[i] + B * c_in[i]

### Adding Numba Compilation

In [17]:
from numba import njit

@njit
def numba_calc_rho(cdot, B):
    rho = np.sqrt(1 + (B * B) * ((cdot * cdot) - 1)) - (B * cdot)
    return rho

@njit
def numba_integrate_context(c, c_in, B, c_ind):
    cdot = 0
    
    for i in range(c_ind[0], c_ind[1]):
        cdot += c[i] * c_in[i]
    rho = numba_calc_rho(cdot, B) # you can't mix numba with base python, so we use the numba version of calc_rho

    for i in range(c_ind[0], c_ind[1]):
        c[i] = rho * c[i] + B * c_in[i]

### Speed Comparison
We run each function to confirm they work before our speed test, but also because the first run of a JIT-compiled function is necessarily slower than subsequent runs because of the compilation step.
The cost of compilation step is negligible when the function is called many times, but it can be significant when the function is only called once.
Even in this case, compilation results can be cached to ensure this cost is only paid once, similar to how Cython compiles code just once.
We don't do any demonstration of caching here, though.
We use the `timeit` module to time each function call, allowing the module to configure the number of loops and repetitions to get a good estimate of the time required to run each function.

In [18]:
c1 = np.array([0.0, 0.09128709, 0.18257419, 0.27386128, 0.36514837, 0.8660254])
c2 = np.array([0.15655607, 0.24875946, 0.34096284, 0.43316622, 0.52536961, 0.57767384])
B = 0.5
c_ind = np.array([0, 6], dtype=np.int32)

integrate_context(c1, c2, B, c_ind)
cython_integrate_context(c1, c2, B, c_ind)
numba_integrate_context(c1, c2, B, c_ind)

In [19]:
%timeit integrate_context(c1, c2, B, c_ind)

5.77 µs ± 185 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [20]:
%timeit cython_integrate_context(c1, c2, B, c_ind)

992 ns ± 28.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [21]:
%timeit numba_integrate_context(c1, c2, B, c_ind)

556 ns ± 14.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


This speed comparison shows that the Numba-compiled Python code is even significantly faster than the Cython implementation, and that both are several times than the base Python implementation.
Outcomes like these are why I think Numba is worth considering as an alternative to Cython for speeding up most of the code we use to do much of our research.